Regression 문제는 숫자 값을 예측하려고 할 때마다 사용한다. 일반적인 예로 가격 예측(주택, 주식 등), 입원 기간 예측(병원에 입원한 환자의 경우), 수요 예측(소매 판매) 등이 있다.   
예를 들어 면적(피트제곱)과 연식(년)에 따라 주택 가격(달러)을 추정한다고 가정합니다. 주택 가격 예측 모델을 개발하기 위해서는 각 주택의 판매 가격, 면적, 연식 등 판매로 구성된 데이터가 먼저 있어야한다.  
이때, 머신 러닝 용어로 데이터 세트를 **training set**라고 하고  
각 행(하나의 판매에 해당하는 데이터를 포함)을 **example(또는 data point , instance, sample)** 라고 합니다.   
우리가 예측하려고 하는 것(가격)을 **label(또는 target)** 이라고 합니다.  
예측의 기반이 되는 변수(연식 및 면적)를 **feature (또는 covariates**)이라고 합니다.

Linear regression은 regression문제를 해결하기 위한 간단하고 가장 유명한 표준 도구입니다.  
Linear regression은 몇가지 가정에서 나오는데,  
첫째로, feature x와 target y의 관계가 approximately linear라고 가정합니다. 즉, E[Y | X = **x**]의 조건부 평균은 feature **x**의 weighted sum으로 표현될 수 있습니다.  
둘째로, observation noise로 인해 기대값과 target값이 편차가 있을 수 있는데, 이는 가우시안 분포를 따라 정규화가 되면 이런 노이즈에서도 잘 동작 한다고 가정합니다.



일반적으로 데이터 세트의 examples 수를 n으로 표시합니다. 위 첨자를 사용하여 샘플과 대상을 열거하고 아래 첨자를 사용하여 좌표를 인덱싱합니다.  
$x_j^{(i)}$에서 i-th는 샘플, j-th는 coordinate

### 3.1.1.1 Model
핵심은 어떻게 feature가 target의 추정치로 변환 될 수 있는가? 이다. 


$ price = w_{area} * area + w_{age} *age + b $

여기서 w들은 weight이고, b는 bias, offset, intercept라고 불립니다. bias는 모든 features가 0일때, 추정 값으로 판별될 수 있습니다. 면적이 0인 신축 주택은 절대 볼 수 없겠지만, bias는 여전히 필요합니다. 우리가 특징의 모든 linear function을 표현할 수 있게 해주기 때문에(원점을 통과하는 선으로 제한하는 것과 대조적으로). 엄밀히 말해서, 식(3.1.1)은 weighted sum을 통한 linear transform과 bias를 추가한 translation이 결합된 input features의 affine transformation입니다. 주어진 데이터 세트에서 우리의 목표는 평균적으로 우리 모델의 예측이 데이터에서 관찰된 실제 가격에 가능한 한 가깝게 일치하도록 가중치 w와 편향 b를 선택하는 것입니다.